In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import pandas as pd

from scipy.interpolate import interp1d
import glob
import os
import string
from PIL import Image

In [ ]:
def read_density(cont, Nx, Nz):
    '''
    Read density data from density_step.txt to extract interfaces
    '''

    Rho = np.loadtxt("density_"+str(cont)+".txt",skiprows=2, unpack=True, comments="P")
    Rho = np.reshape(Rho, (Nz, Nx))

    return Rho

def extract_interface(z, Z, Nx, Rhoi, rho):
    '''
    Extract interface according to a given density
    '''
    topo_aux = []

    for j in np.arange(Nx):
        topoi = interp1d(z, Rhoi[:,j]) #return a "function" of interpolation to apply in other array
        idx = (np.abs(topoi(Z)-rho)).argmin()
        topo = Z[idx]
        topo_aux = np.append(topo_aux, topo)

    return topo_aux

def extract_topography(step):
    # Read surface topography
    topo= np.loadtxt(f"sp_surface_global_{step}.txt", skiprows=2, unpack=True, comments="P")
    topo = topo/1000
    return topo


In [ ]:
with open("param.txt", "r") as f:
    line = f.readline()
    line = line.split()
    Nx = int(line[2])
    line = f.readline()
    line = line.split()
    Nz = int(line[2])
    line = f.readline()
    line = line.split()
    Lx = float(line[2])
    line = f.readline()
    line = line.split()
    Lz = float(line[2])

data = [Nx, Nz, Lx, Lz]

xi = np.linspace(0, Lx / 1e3, Nx)
zi = np.linspace(-Lz / 1e3, 0, Nz)

xx, zz = np.meshgrid(xi, zi)

thickness_air = 40.0

In [ ]:
# Initial, final and step size to plot the graphs

step_initial = 0
step_final = 10000

d_step = 20

In [ ]:
t, vel, vis, top = [], [], [], [] # surface topography

model_name = '--'

for cont in range(step_initial, step_final + d_step, d_step): 
    # Read time
    time = np.loadtxt("time_" + str(cont) + ".txt", dtype="str")
    time = time[:, 2:]
    time = time.astype("float")
    time = time[0]
    
    t = np.append(t, time)
    
    # Read velocity
    veloc = np.loadtxt(
        "velocity_" + str(cont) + ".txt",
        delimiter=" ",
        comments="P",
        skiprows=2,
    )
    
    velocz = veloc[1::2]
    velocz = np.reshape(velocz, (Nx, Nz), order="F")
    velocz = np.transpose(velocz)
    vzmean = np.mean(velocz[80:200,730:800])
    
    vel = np.append(vel, vzmean)
    
    # Read viscosity
    visc = np.loadtxt(
        "viscosity_" + str(cont) + ".txt",
        delimiter=" ",
        comments="P",
        skiprows=2,
    )
    
    visc = visc[::]
    visc = np.reshape(visc, (Nx, Nz), order="F")
    visc = np.transpose(visc)
    viscmean = np.mean(np.log10(visc[130:200,600:800]))
    
    vis = np.append(vis, viscmean)
    
    # Calculate topography
    Nx, Nz, Lx, Lz = data
    
    z_mean = 40.0 #np.mean(topo[condx])

    condx = (xi >= 200) & (xi <= 300)
    
    ### checar onde que vamos calcular a media
    topo = extract_topography(cont)
    topo -= z_mean #h_air
    condx = (xi >= 200) & (xi <= 300)
    mean = np.mean(topo[condx])
    topo += np.abs(mean)
    
    topo_lump = np.mean(topo[550:660])
    top = np.append(top, topo_lump)

In [ ]:
# Velocity

fig, (ax1, ax2) = plt.subplots(2, 1, sharex=True)
fig.suptitle(f'Correlation between topography and EDC - {model_name}')
ax1.plot(t/1e6,vel*(60*60*24*365)*1e2,'r')
ax1.grid('-k', alpha=0.7)
ax1.set_ylim(-1.5, 1.0)
ax1.set_ylabel('Velocity [cm/yr]')
ax1.tick_params(axis='y')
ax1.tick_params(bottom = False)

ax2.plot(t/1e6,top, label='surface')
ax2.grid('-k', alpha=0.7)
ax2.set_xlim(t[0]/1e6, t[-1]/1e6)
ax2.set_ylim(-0.3, 0.1)
ax2.set_ylabel("Depth [km]")
ax2.set_xlabel("Time [Myr]")
# ax2.legend(loc='upper right')

plt.savefig(f'{model_name}_vel.png', format='png')

In [ ]:
# Viscosity

fig, (ax1, ax2) = plt.subplots(2, 1, sharex=True)
fig.suptitle(f'Correlation between topography and mantle viscosity - {model_name}')
ax1.plot(t/1e6, vis, color='orange')
ax1.grid('-k', alpha=0.7)
ax1.set_ylabel('Viscosity [$Log_{10}$(Pa s)]')
ax1.tick_params(axis='y')
ax1.tick_params(bottom = False)

ax2.plot(t/1e6,top, label='surface')
ax2.grid('-k', alpha=0.7)
ax2.set_xlim(t[0]/1e6, t[-1]/1e6)
ax2.set_ylim(-0.3, 0.1)
ax2.set_ylabel("Depth [km]")
ax2.set_xlabel("Time [Myr]")
# ax2.legend(loc='upper right')

plt.savefig(f'{model_name}_visc.png', format='png')